# HW3: Where is the Röstigraben?

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as rq

## Data Wrangling

In [ ]:
df = pd.read_csv('data/P3_GrantExport.csv', delimiter=';', 
                 index_col=0, na_values=['Nicht zuteilbar - NA', 'data not included in P3'],
                 parse_dates=["Start Date", "End Date"])

In [ ]:
no_amount = df[['Approved Amount']].isnull().mean()[0]
print('{:d}% of \'Approved Amount\' is NaN'.format(int(100*no_amount)))

In [ ]:
universities = df['University'].str.split(' - ', expand=True).rename(columns={0:'Name', 1:'Abbr'})
universities.dropna(axis=0,inplace=True)
universities.drop_duplicates(inplace=True)
universities.size

In [ ]:
cantons = pd.read_csv('data/cantons_codes.csv', header=None, names=['Name'])
cantons.head()

In [ ]:
# Making the canton
universities['Canton'] = universities['Abbr'].apply(lambda x: x if x in cantons.values else None)
universities['Canton'].isnull().sum()

In [ ]:
# Use Google to fill the missing values
from keys import GOOGLE_KEY
import json

def search_canton(json):
    if json['results'] and 'address_components' in json['results'][0]:
            for cs in json['results'][0]['address_components']:
                if 'short_name' in cs:
                    name = cs['short_name']
                    if name in cantons.values:
                        return name
    return None

URL = 'https://maps.googleapis.com/maps/api/geocode/json'
for idx in universities.index:
    if not universities.loc[idx]['Canton']:
        name = universities.loc[idx]['Name']
        r = rq.get(URL, params={'key':GOOGLE_KEY,'address':name,'region':'CH'})
        j = json.loads(r.text)
        key = search_canton(j)
        universities.loc[idx]['Canton'] = key

universities['Canton'].isnull().sum()

In [ ]:
universities[universities['Canton'].isnull()]

In [ ]:

r = rq.get(URL2, params={'key':GOOGLE_KEY,'address':'IHEID','region':'CH'})
j = json.loads(r.text)
j
